<a href="https://colab.research.google.com/github/suhedacilek/MachineLearningAlgorithms/blob/main/PyTorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch ile MNIST veri kümesi üzerinde MLP ve CNN eğitme 

![MNIST örnekleri](https://www.researchgate.net/profile/Stefan_Elfwing/publication/266205382/figure/fig5/AS:267913563209738@1440886979379/Example-images-of-the-ten-handwritten-digits-in-the-MNIST-training-set.png)

- 10 sınıf
- 60 bin eğitme görüntüsü
- 10 bin test görüntüsü
- Her bir görüntü 28x28 piksel, tek renkli

In [ ]:
#@title Kullanılacak modülleri ekle
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [ ]:
# "GPU varsa PyTorch onu kullansın, yoksa CPU kullansın" ayarı
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#@title Veri kümesini indir
egitim_verisi = datasets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_verisi = datasets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [ ]:
# ToTensor() dönüşümü hakkında:   Boyutlar arası dönüşümü sağlar.

# PyTorch aglarinin bekledigi girdi boyutlari: N*C*H*W 
# N: yigin boyutu (batch size)
# C: kanal sayisi (channel size)
# H: yukseklik (height)
# W: genislik (width)

# Normalde bir görüntü H*W*C şeklinde olur. 
# ToTensor() dönüşümü renk kanalını başa alıyor. 

In [ ]:
#@title Veri yükleyicileri tanımla
batch_size = 100
egitim_veri_yukleyici = torch.utils.data.DataLoader(dataset = egitim_verisi,
                                             batch_size = batch_size,
                                             shuffle = True) #Karıştırır.

test_veri_yukleyici = torch.utils.data.DataLoader(dataset =  test_verisi ,
                                      batch_size = batch_size, 
                                      shuffle = False) #Burada shuffle etsek te olur etmesekte olur.

In [ ]:
#@title MLP ağını tanımla

class MLP(nn.Module):
  def __init__(self,girdi_boyutu, sakli_katman_noron_sayisi, sinif_sayisi):       #kullanacağımız katmanları tanımlarız.
    super(MLP,self).__init__()
    self.fc1 = nn.Linear(girdi_boyutu, 32)   #saklı katman sayısını alan bir fonksiyon. 
    self.fc2 = nn.Linear(32,64)   #100 görüntü x 10 luk bir görüntü olacak.Batch size 100, her bir görüntü 10 ile çarpılacak.
    self.fc3 = nn.Linear(64,sinif_sayisi)
    self.relu = nn.ReLU()  
  
  def forward(self,girdi):
    x = self.relu(self.fc1(girdi))
    print(x.shape)
    x = self.relu(self.fc2(x))
    print(x.shape)
    x= self.fc3(x)
    print(x.shape)
   
    return x

In [ ]:
#@title Bir model örneği yaratalım

girdi_boyutu = 28*28   #fc1 de 28x28 lik oldugu için 784 özellik girdi.Saklı katmana girince 1x64 lük , çıktı ise 1x10 luk.
sakli_katman_noron_sayisi = 64     
sinif_sayisi = 10

net = MLP(girdi_boyutu, sakli_katman_noron_sayisi, sinif_sayisi)
net.to(device)  #device GPU demek.

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
#@title Kayıp fonksiyonu ve optimizasyon yöntemini tanımla
kayip_fonk = nn.CrossEntropyLoss()   #her bir görüntü için elde ettiğimiz 10 skoru 0 1 arasına çeker ve doğru etiketi[0000...01..] oluşturur. Doğru olan skor 1 gerisi 0 dır. 1x10 luk çıktıyı doğru etiketine benzetmeye çalışırız.
optimizasyon_yontemi = torch.optim.Adam( net.parameters(), lr=1.e-3) #stocastic optimizasyon yöntemlerinden hangisini kullnacağımızı seçiyor.

In [ ]:
#@title Modelimizi eğitelim

epoch_sayisi = 5
for epoch in range(epoch_sayisi):  #epoch anlamı veri kümesi üzerinden bir tam geçiş yap demektir.
  for i ,(goruntuler,etiketler) in enumerate(egitim_veri_yukleyici): #batchler üzerinden yapılan bir döngü.
    goruntuler = goruntuler.view(-1,28*28).to(device)  ##-1 satır sayısını sen hesapla demek ama sütün sayısı 28x28 olacaktır.
    etiketler = etiketler.to(device)
    
    optimizasyon_yontemi.zero_grad()  #gradientleri 0 lamak gibi birşey. 
    ciktilar = net(goruntuler)  #100 tane düzleştirilmiş görüntü ağımıza girdi ve çıktı olarak bize 100x10 luk bir çıktı verdi.
    loss = kayip_fonk(ciktilar, etiketler)  #modelin ürettiği.
    loss.backward()  #türevi hesapladı.
    optimizasyon_yontemi.step() #optim.Adam olarak yukarıda tanımladığımız şeye göre adım atmasını sağlıyor. 
    
    if (i+1) % batch_size == 0:   #100 iterasyonda 1 loss fonksiyonunu bastır.
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, epoch_sayisi, i+1, len(egitim_verisi)//batch_size, loss.item()))

torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size

In [ ]:
#@title Modeli test kümesi üzerinde çalıştır

dogru = 0
toplam = 0
for goruntuler,etiketler in test_veri_yukleyici:
  goruntuler = goruntuler.view(-1,28*28).to(device)
  etiketler = etiketler.to(device)
  
  cikti = net(goruntuler) #modeli görüntüler üzerinden çalıştırdık 10 luk skor değerleri verdi.
  _, tahmini_etiketler = torch.max(cikti,1) #10 luk tahminlerdeki en yüksek sayıyı verir.
  dogru += (tahmini_etiketler == etiketler).sum() #100 örenketen kaçı doğrudur.
  toplam += etiketler.size(0)

print('Test kümesi üzerinde doğruluk: %.3f %%' %((100*dogru)/(toplam+1))) 

torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size([100, 32])
torch.Size([100, 64])
torch.Size([100, 10])
torch.Size

# Şimdi de bir CNN modeli oluşturup, eğitelim

In [ ]:
#@title CNN ağını tanımla
class CNN(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #monocrom resimlerle çalıştığımız için bu ilk parametre 1, renkli olsaydı 3 olacaktı.
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        #self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 64)
        self.fc2 = nn.Linear(64, 10)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))  
        x = x.view(-1, 320)
        x = self.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x


# bir ağ örneği yaratalım: 
net = CNN().to(device)

In [ ]:
#@title CNN eğitimi
kayip_fonk = nn.CrossEntropyLoss()
optimizasyon_yontemi = torch.optim.Adam( net.parameters(), lr=1.e-3)

epoch_sayisi = 5
for epoch in range(epoch_sayisi):
  for i ,(goruntuler,etiketler) in enumerate(egitim_veri_yukleyici):
    #MLP'de soyle yapıyorduk: 
    # goruntuler = goruntuler.view(-1,28*28).to(device)  Cnn ler image üzerinden doğrudan çalıştıkları için düzleştirmeye gerek duymuyoruz.
    goruntuler = goruntuler.to(device)
    etiketler = etiketler.to(device)
    
    optimizasyon_yontemi.zero_grad()
    ciktilar = net(goruntuler)
    loss = kayip_fonk(ciktilar, etiketler)
    loss.backward()
    optimizasyon_yontemi.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, epoch_sayisi, i+1, len(egitim_verisi)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.5139
Epoch [1/5], Step [200/600], Loss: 0.2585
Epoch [1/5], Step [300/600], Loss: 0.3115
Epoch [1/5], Step [400/600], Loss: 0.1910
Epoch [1/5], Step [500/600], Loss: 0.0937
Epoch [1/5], Step [600/600], Loss: 0.1380
Epoch [2/5], Step [100/600], Loss: 0.0693
Epoch [2/5], Step [200/600], Loss: 0.2845
Epoch [2/5], Step [300/600], Loss: 0.0387
Epoch [2/5], Step [400/600], Loss: 0.1233
Epoch [2/5], Step [500/600], Loss: 0.1363
Epoch [2/5], Step [600/600], Loss: 0.1116
Epoch [3/5], Step [100/600], Loss: 0.1359
Epoch [3/5], Step [200/600], Loss: 0.0448
Epoch [3/5], Step [300/600], Loss: 0.0499
Epoch [3/5], Step [400/600], Loss: 0.0605
Epoch [3/5], Step [500/600], Loss: 0.0426
Epoch [3/5], Step [600/600], Loss: 0.0410
Epoch [4/5], Step [100/600], Loss: 0.0458
Epoch [4/5], Step [200/600], Loss: 0.0421
Epoch [4/5], Step [300/600], Loss: 0.0967
Epoch [4/5], Step [400/600], Loss: 0.0729
Epoch [4/5], Step [500/600], Loss: 0.0215
Epoch [4/5], Step [600/600], Loss:

In [ ]:
#@title CNN modelini test kümesi üzerinde çalıştır

dogru = 0
toplam = 0
for goruntuler,etiketler in test_veri_yukleyici:
  goruntuler = goruntuler.to(device)
  etiketler = etiketler.to(device)
  
  cikti = net(goruntuler)
  _, tahmini_etiketler = torch.max(cikti,1)
  dogru += (tahmini_etiketler == etiketler).sum()
  toplam += etiketler.size(0)

print('Test kümesi üzerinde doğruluk: %.3f %%' %((100*dogru)/(toplam+1)))

Test kümesi üzerinde doğruluk: 98.000 %
